In [ ]:
import colamatch as clm
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
import h5py
from brainmap.transformation import transformation
%matplotlib notebook
import logging
import sys
logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format='%(asctime)s %(name)s [%(levelname)s]:%(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')

# Artificial Testset

## Define filenames and read images and landmarks

In [ ]:
s_fixed = 1
s_moving = 2
fixed_file = "../data/section_%s.tif"%(str(s_fixed).zfill(4))
moving_file = "../data/section_%s.tif"%(str(s_moving).zfill(4)) 
fixed_landmarks = "../data/section_%s.landmarks.txt"%(str(s_fixed).zfill(4))
moving_landmarks = "../data/section_%s.landmarks.txt"%(str(s_moving).zfill(4))

In [ ]:
# read landmarks and images
l_fixed = np.loadtxt(fixed_landmarks).astype('int')
l_moving = np.loadtxt(moving_landmarks).astype('int')
img_fixed = cv2.imread(fixed_file,0)
img_moving = cv2.imread(moving_file,0)
print(len(l_fixed), len(l_moving))

## Perform constellation matching

In [ ]:
num_samples = 10000
sampler_fixed = clm.RandomSampler(len(l_fixed), 4, num_samples)
sampler_moving = clm.RandomSampler(len(l_moving), 4, num_samples)
start = time.time()
matches = clm.match(l_fixed, l_moving, sampler_fixed, sampler_moving, radius=0.1, lamda=2, ransac=0.01)
print("runtime for num_samples=%s: %f" % (num_samples,time.time()-start))

In [ ]:
cmap = plt.cm.get_cmap("hsv", len(matches))
fig,axs = plt.subplots(1,2)
axs[0].imshow(img_fixed, cmap='gray')
axs[1].imshow(img_moving, cmap='gray')
axs[0].plot(l_fixed[:,0], l_fixed[:,1],'w+',ms=15)
axs[1].plot(l_moving[:,0], l_moving[:,1],'w+',ms=15)
axs[0].axis('off')
axs[1].axis('off')
for i,candidate in enumerate(matches): 
    axs[0].plot(candidate[0,0], candidate[0,1], c=cmap(i), marker='o')
    axs[1].plot(candidate[1,0], candidate[1,1], c=cmap(i), marker='o')
fig.show()

# Match real vessel detections

### Load detected (& preregistrered) landmarks, ROI coordinates and images

In [ ]:
### PARAMETERS ###
brainid = "B20"
structure = "V1_l"
fixed = 1529
moving = 1530
numLandmarks = 100   # use only best x landmarks for constellation matching
use_preregistration = True

In [ ]:
# read detected landmarks in fixed and moving image
l_fixed = h5py.File("../data/{}_{}_{}_vessels.h5".format(brainid,structure,fixed))["data"][:numLandmarks,:2]
l_moving = h5py.File("../data/{}_{}_{}_vessels.h5".format(brainid,structure,moving))["data"][:numLandmarks,:2]
print(l_fixed.shape[0], l_moving.shape[0])
if use_preregistration:
    # transform moving landmarks with preregistration
    prereg_file = "../data/{}_{}_transformation.json".format(brainid,moving)
    prereg = transformation.Transformation.from_json(prereg_file)
    prereg_inverse = transformation.Transformation.from_json(prereg_file, inverse=True)
    l_moving = prereg.apply_to_coords(l_moving, 0.001, 0.001)

In [ ]:
# load images and ROIs for plotting
scale = 0.1
roifile = h5py.File("../data/{}_{}_rois.h5".format(brainid,structure))
roi_fixed = roifile["{}/roi".format(fixed)][:]
roi_moving = roifile["{}/roi".format(moving)][:]
roifile.close()
img_fixed = cv2.imread("../data/{}_{}_{}_scale0.1.tif".format(brainid,structure,fixed),0)
img_moving = cv2.imread("../data/{}_{}_{}_scale0.1.tif".format(brainid,structure,moving),0)

## Constellation Matching

### Parameters for constellation matching:
* **num_samples**: Number of hashes added to KDTree for fixed and moving landmarks (depends on #landmarks -> more samples needed for increasing number of landmarks)
* **lambda**: Weight for absolute landmark coordinates in hash code (should be >1; 2 works good)
* **ransac**: Hessian threshold for homography ransac on matched normalized landmark coordinates (0.01 works good)
* **radius**: Radius for finding similar hashes in KDTree, could be calculated in match()-function with
$$radius = \sqrt{(\lambda*maxDist*scale)^2*3 + \Delta relativ\_quad\_positions^2 * 4}$$, 
where $scale$ = scale used in normalization of landmarks (known in match() function), $maxDist$ = maximal distance between corresponding pixels in fixed and moving image (after preregistration, ~250), $\Delta relativ\_quad\_positions$ = maximal allowed distance between cx, cy, dx and dy of similar quads (0.0015 seems to be a good value here). Is it necessary to adapt radius in relation to the ratio of #landmarks/num_samples?

In [ ]:
# constellation matching
num_samples = 50000
c2_fixed = clm.RandomSampler(len(l_fixed), 4, num_samples)
c2_moving = clm.RandomSampler(len(l_moving), 4, num_samples)
start = time.time()
matches = clm.match(l_fixed, l_moving, c2_fixed, c2_moving, radius=0.025, lamda=2, ransac=0.01) 
print("runtime for num_samples=%s: %f" % (num_samples,time.time()-start))
if use_preregistration and len(matches) > 0:
    # transform moving match coordinates back to original moving image space: 
    matches_moving = prereg_inverse.apply_to_coords(matches[:,1], 0.001, 0.001)
    matches = np.hstack((matches[:,0], matches_moving)).reshape(-1,2,2)
print("Found %d matches"%len(matches))

### Plot matches

In [ ]:
# plot landmarks and matches
if use_preregistration:
    l_moving_orig = prereg_inverse.apply_to_coords(l_moving, 0.001, 0.001)
else: 
    l_moving_orig = l_moving
cmap = plt.cm.get_cmap("hsv", len(matches))
fig,axs = plt.subplots(2,1)
axs[0].imshow(img_fixed, cmap='gray')
axs[1].imshow(img_moving, cmap='gray')
# plot (landmarks - roi-offset) * scale according to downscaled images (0.1)
axs[0].plot((l_fixed[:,0]-roi_fixed[0,0])*scale, (l_fixed[:,1]-roi_fixed[0,1])*scale,'w+',ms=10)
axs[1].plot((l_moving_orig[:,0]-roi_moving[0,0])*scale, (l_moving_orig[:,1]-roi_moving[0,1])*scale,'w+',ms=10)
axs[0].axis('off')
axs[1].axis('off')
for i,match in enumerate(matches): 
    # plot matched coordinates - roi-offset * scale according to downscaled images (0.1)
    axs[0].plot((match[0,0]-roi_fixed[0,0])*scale, (match[0,1]-roi_fixed[0,1])*scale, c=cmap(i), marker='o')
    axs[1].plot((match[1,0]-roi_moving[0,0])*scale, (match[1,1]-roi_moving[0,1])*scale, c=cmap(i), marker='o')
fig.show()